In [1]:
import torch
import matplotlib.pyplot as plt
import BVP_function as bvp
import numpy as np
import deepxde as dde

import seaborn as sns
sns.set_style("whitegrid") # better looking plots

print("Using GPU:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

from importlib import reload
reload(bvp)

Using backend: pytorch



Using GPU: False


<module 'BVP_function' from 'c:\\Users\\u0150568\\OneDrive - KU Leuven\\Code\\public github\\pinn-cm\\inverse\\BVP_function.py'>

In [19]:
reload(bvp)

lmbd = 1.0
lmbd_trainable = dde.Variable(lmbd - 0.2)
mu = 0.5
mu_trainable = dde.Variable(mu + 0.2)
Q = 4.0

domain = np.array([[0.0, 1.0], [0.0, 1.0]])
geom = dde.geometry.Rectangle([0, 0], [1, 1])

phy_params = {'lmbd': lmbd, 'mu': mu, 'Q': Q}
phy_params_trainable = {'lmbd': lmbd_trainable, 'mu': mu_trainable, 'Q': Q}

#net parameters
net_type = ['Unet','USnet'][1]
n_layers = 4
size_layers = 50
activation = ['tanh','ReLU','Sigmoid'][0]

#loss parameters
loss_type = ['pde','energy'][0] #the energy loss appears to crash LBFGS and provide less accurate results with Adam. PDE loss is therefore used in the following
num_domain = 50**2
train_distribution = ['uniform','pseudo','LHS','Halton','Hammersley','Sobol'][4]

#boundary conditions
bc_type = ['soft','hard'][1]
num_boundary = 50

#optimizer
optimizers = ["adam","L-BFGS"]
learning_rates = [1e-3,None]
iterations = [3000,None]

config = {'net_type':net_type,'n_layers':n_layers,'size_layers':size_layers,'activation':activation,
        'loss_type':loss_type,'num_domain':num_domain,'train_distribution':train_distribution,
        'bc_type':bc_type,'num_boundary':num_boundary,
        'optimizers':optimizers,'learning_rates':learning_rates,'iterations':iterations}  

net_exact = bvp.set_exact_solution(net_type, phy_params,lib='torch')

model, net_wrong_elasticity, pde_net, energy_net, mat_net = bvp.model_setup(geom, config, phy_params_trainable)

In [20]:
trainable_variables = [lmbd_trainable, mu_trainable]
variable = dde.callbacks.VariableValue(trainable_variables, period=100, filename="elasticity_param.dat")

model.compile("adam", lr=0.001, external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=5000,display_every=1000, callbacks=[variable])

model.compile("adam", lr=0.0001, external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=5000,display_every=1000, callbacks=[variable])

model.compile("adam", lr=0.0001, external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=5000,display_every=1000, callbacks=[variable])

# # train lbfgs
# model.compile("L-BFGS", external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
# losshistory, train_state = model.train(callbacks=[variable])
print(f"lambda:{lmbd_trainable} ; mu: {lmbd_trainable}")

Compiling model...
'compile' took 0.000423 s

Training model...

Step      Train loss                        Test loss                         Test metric   
0         [2.23e+03, 2.65e+02, 2.10e+03]    [2.22e+03, 2.64e+02, 2.17e+03]    [1.14e+00]    
200       [9.84e+02, 9.18e+01, 5.18e+01]    [9.84e+02, 9.07e+01, 4.67e+01]    [9.37e-01]    
400       [1.68e+02, 1.33e+01, 2.77e+01]    [1.61e+02, 1.31e+01, 2.62e+01]    [7.02e-01]    
600       [4.77e+01, 9.49e+00, 1.73e+01]    [4.48e+01, 8.87e+00, 1.59e+01]    [6.37e-01]    
800       [1.92e+01, 4.86e+00, 1.03e+01]    [1.76e+01, 4.34e+00, 9.34e+00]    [6.10e-01]    
1000      [9.77e+00, 2.93e+00, 6.80e+00]    [8.66e+00, 2.49e+00, 6.05e+00]    [5.99e-01]    
1200      [5.84e+00, 1.92e+00, 4.79e+00]    [5.02e+00, 1.57e+00, 4.20e+00]    [5.97e-01]    
1400      [3.65e+00, 1.30e+00, 3.55e+00]    [3.08e+00, 1.03e+00, 3.07e+00]    [5.97e-01]    
1600      [2.36e+00, 9.89e-01, 2.69e+00]    [1.95e+00, 7.64e-01, 2.29e+00]    [5.98e-01]    
1800 

KeyboardInterrupt: 